This is a notebook that contains a class `Bit` that when we perform Python logical operations on it such as `&` , `|` , `~` (AND,OR,NOT) it remembers them and helps build a "computation graph" for them.

We can then use this to either write a formula or draw a circuit for general python code that works on bits

## Class for bit operations

Inspired by the `Value` class of Karpathy, which in turn is inpired by Pytorch's `Tensor` see https://windowsontheory.org/2020/11/03/yet-another-backpropagation-tutorial/

In [ ]:
class Bit:
  counter = 0
  
  @classmethod
  def uid(cls):
    cls.counter +=1
    return f"t_{cls.counter}"

  def __init__(self,label = None): 
    self.label = self.uid() if label is None else label
    self.program = [self.label]
  
  def op(self,oper, *others):
    operands = [self, *others]
    out = Bit()
    out.program = [oper, *[bit.program for bit in operands]]
    return out

  def __and__(self,other): return self.op("∧", other)
  def __or__(self,other): return self.op("∨", other)
  def __invert__(self): return self.op("¬")

def bits(n):
  return [Bit(f"x_{i}") for i in range(n)]

In [ ]:
def formula(P):
  if len(P)==1:
    return P[0]
  if len(P)==2:
    return P[0] + formula(P[1])
  if len(P)==3:
    return f"({formula(P[1])} {P[0]} {formula(P[2])})"
  return f"{P[0]}("+",".join([formula(P[i]) for i in range(1,len(P))])+")"



In [ ]:
a,b,c,d = bits(4)
out = (a&b)|~(c&d)

In [ ]:
s = formula(out.program)
s

'((x_0 ∧ x_1) ∨ ¬(x_2 ∧ x_3))'

In [ ]:
from IPython.display import Math
Math(s)

<IPython.core.display.Math object>

## Evaluate formula on inputs

In [ ]:
def inputs(P):
  if len(P)==1:
    return P[0]
  all_inputs = sum([inputs(P[i]) for i in range(1,len(P))],[])
  return list(set(all_inputs))


def evalp(P,D):
  if len(P)==1:
    return D[P[0]]
  if P[0]=='¬':
    return 1-evalp(P[1],D)
  if P[0]=='∧':
    return evalp(P[1],D)*evalp(P[2],D)
  if P[0]=='∨':
    return 1-(1-evalp(P[1],D))*(1-evalp(P[2],D))
  

  

In [ ]:
evalp(out.program, {"x_0":0, "x_1":1, "x_2":1 , "x_3":1 })

0

## Showing some operations

In [ ]:
import itertools
from IPython.display import Markdown, display, Math

def table(f,n):
  """Generate truth table of a function"""
  m = max(n+2,len(f.__name__)+4)
  res = "x".ljust(m) + " | " + f"{f.__name__}(x)".ljust(m) 
  res += "\n" + "-"*m+"-|-"+ "-"*m 
  
  for x in itertools.product([0,1],repeat=n):
    s = "".join([str(c) for c in x])
    res += "\n"+ s.ljust(m) + " | " + str(f(*x)).ljust(m)
  res +="\n"
  return Markdown(res)

In [ ]:
def xor2(a,b): return (a & ~b) | (~a & b)

table(xor2,2)

x        | xor2(x) 
---------|---------
00       | 0       
01       | 1       
10       | 1       
11       | 0       


In [ ]:
Y = xor2(*bits(2))
Math(formula(Y.program)) 

<IPython.core.display.Math object>

In [ ]:
def xor(*L): return xor2(*L) if len(L)==2 else xor2(xor(*L[:-1]),L[-1])
Math(formula(xor(*bits(3)).program))

<IPython.core.display.Math object>

In [ ]:
def maj(a,b,c): return (a & b) | (b&c) | (a&c)
def onebitadd(a,b,c):
  return xor(a,b,c) , maj(a,b,c)

table(onebitadd,3)

x             | onebitadd(x) 
--------------|--------------
000           | (0, 0)       
001           | (1, 0)       
010           | (1, 0)       
011           | (0, 1)       
100           | (1, 0)       
101           | (0, 1)       
110           | (0, 1)       
111           | (1, 1)       


In [ ]:
def zero(a): return a & ~a
table(zero,1)


x        | zero(x) 
---------|---------
0        | 0       
1        | 0       


In [ ]:
def maj(a,b,c): return (a & b) | (b&c) | (a&c)
def zero(a): return a & ~a
def xor2(a,b): return (a & ~b) | (~a & b)
def xor(*L): return xor2(*L) if len(L)==2 else xor2(xor(*L[:-1]),L[-1])

In [ ]:
def add(A,B): 
  """Add two binary numbers, given as lists of bits"""
  Y = []
  carry = zero(A[0]) # initialize carry to 0
  for i in range(len(A)): # compute i-th digit of output
    y = xor(A[i],B[i],carry) # xor function
    carry = maj(A[i],B[i],carry) # majority function
    Y.append(y)
  Y.append(carry)
  return Y


table(lambda a,b,c,d: add([b,a],[d,c])[::-1],4)

x            | <lambda>(x) 
-------------|-------------
0000         | [0, 0, 0]   
0001         | [0, 0, 1]   
0010         | [0, 1, 0]   
0011         | [0, 1, 1]   
0100         | [0, 0, 1]   
0101         | [0, 1, 0]   
0110         | [0, 1, 1]   
0111         | [1, 0, 0]   
1000         | [0, 1, 0]   
1001         | [0, 1, 1]   
1010         | [1, 0, 0]   
1011         | [1, 0, 1]   
1100         | [0, 1, 1]   
1101         | [1, 0, 0]   
1110         | [1, 0, 1]   
1111         | [1, 1, 0]   


In [ ]:
X1 = [Bit("A_0"),Bit("A_1")]
X2 = [Bit("B_0"),Bit("B_1")]
Y = add(X1,X2)
Math(formula(Y[0].program))

<IPython.core.display.Math object>